In [91]:
import os
import sys
import numpy as np
import matplotlib as mpl
import pandas as pd

# run installed version of flopy or add local path
try:
    import flopy
except:
    fpth = os.path.abspath(os.path.join('..', '..'))
    sys.path.append(fpth)
    import flopy

print(sys.version)
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('pandas version: {}'.format(pd.__version__))
print('flopy version: {}'.format(flopy.__version__))

3.7.7 (default, Mar 10 2020, 15:43:33) 
[Clang 11.0.0 (clang-1100.0.33.17)]
numpy version: 1.18.3
matplotlib version: 3.2.1
pandas version: 1.0.3
flopy version: 3.3.0


### Model inputs

In [92]:
# load the model
project_name = "UMONS_SmallModel_SteadyState"
model_ws = os.path.join('.', "SteadyState")
ml = flopy.modflow.Modflow.load(
    project_name+".mfn", 
    model_ws=model_ws, 
    verbose=True,
    check=False, 
    exe_name="mf2000"
)


Creating new model with name: UMONS_SmallModel_SteadyState
--------------------------------------------------

Parsing the namefile --> ./SteadyState/UMONS_SmallModel_SteadyState.mfn

--------------------------------------------------
External unit dictionary:
{7: filename:./SteadyState/UMONS_SmallModel_SteadyState.obs, filetype:OBS, 8: filename:./SteadyState/UMONS_SmallModel_SteadyState.chob, filetype:CHOB, 9: filename:./SteadyState/UMONS_SmallModel_SteadyState.dis, filetype:DIS, 10: filename:./SteadyState/UMONS_SmallModel_SteadyState.ba6, filetype:BAS6, 11: filename:./SteadyState/UMONS_SmallModel_SteadyState.lpf, filetype:LPF, 12: filename:./SteadyState/UMONS_SmallModel_SteadyState.oc, filetype:OC, 13: filename:./SteadyState/UMONS_SmallModel_SteadyState.rch, filetype:RCH, 16: filename:./SteadyState/UMONS_SmallModel_SteadyState.wel, filetype:WEL, 14: filename:./SteadyState/UMONS_SmallModel_SteadyState.chd, filetype:CHD, 15: filename:./SteadyState/UMONS_SmallModel_SteadyState.pcg, fil

### Load the iBound for each layer

In [93]:
ml.get_package_list()

['DIS', 'CHOB', 'BAS6', 'LPF', 'OC', 'RCH', 'WEL', 'CHD', 'PCG']

In [94]:
bas = ml.get_package('BAS6')
iBound = bas.ibound
iBound.array

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=int32)

In [95]:
from flopy.discretization import StructuredGrid

mg0 = ml.modelgrid
mg0

xll:0.0; yll:0.0; rotation:0.0; units:meters; lenuni:2

In [274]:
xul = 288280
yul = 9112500

xll = xul - 8200
yll = yul - sum(mg0.delr) - 1400

epsg = 31985
proj4 = "+proj=utm +zone=25 +south +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"

modelgrid = StructuredGrid(
    mg0.delc,
    mg0.delr,
    mg0.top,
    mg0.botm,
    mg0.idomain,
    mg0.lenuni,
    epsg=epsg,
    proj4=proj4,
    xoff=xll,
    yoff=yll,
    angrot=-15.5
)

data_folder = 'ReadModelGeometry'
shapefile_name = 'modelGeometry.shp'
fnc = ml.export(os.path.join(data_folder, shapefile_name), modelgrid=modelgrid)

print(xll, yll)

wrote ReadModelGeometry/modelGeometry.shp
280080 9099796.530220032


In [275]:
from pyproj import Proj, transform
inProj = Proj('epsg:'+str(epsg))
outProj = Proj('epsg:4326')

x1,y1 = xll,yll
x2,y2 = transform(inProj,outProj,x1,y1)

In [276]:
from pyproj import Transformer

def transform_point(tf, point):
    xn, yn = tf.transform(point[0], point[1])
    return [xn, yn]

In [277]:
import geojson
import rasterio.features
import numpy.ma as ma

layer = iBound.array[0]
mask = np.array(ma.masked_values(layer, 1, shrink=False), dtype=bool)
mpoly_cells_coordinates=[]
for vec in rasterio.features.shapes(layer, mask=mask):
    mpoly_cells_coordinates.append(geojson.Polygon(vec[0]["coordinates"]))

mpoly_cells_coordinates = mpoly_cells_coordinates[0]

In [278]:
def get_cell_centers(grid, c):

    xcz = grid.xcellcenters
    ycz = grid.ycellcenters

    nx = int(c[0])
    ny = int(c[1])

    return [xcz[ny][nx], ycz[ny][nx]]

mpoly_cell_centers=geojson.utils.map_tuples(lambda c: get_cell_centers(modelgrid, c), mpoly_cells_coordinates)

tf = Transformer.from_crs(epsg, 4326, always_xy=True)
mpoly_coordinates=geojson.utils.map_tuples(lambda c: transform_point(tf, c), mpoly_cell_centers)



In [279]:
from ipyleaflet import Map, GeoJSON

polygon = geojson.Polygon(mpoly_coordinates["coordinates"])
center = mpoly_coordinates["coordinates"][0][0]
mp = Map(center=(center[1], center[0]), zoom=12)
geo_json = GeoJSON(data=mpoly_coordinates, style = {'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1})
mp.add_layer(geo_json)

mp

Map(center=[-8.073346674247246, -34.873848101146756], controls=(ZoomControl(options=['position', 'zoom_in_text…